In [1]:
# !wget https://raw.githubusercontent.com/TensorSpeech/TensorFlowTTS/master/examples/hifigan/conf/hifigan.v2.yaml

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [4]:
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import melgan, hifigan
from malaya_speech.train.model import stft
import malaya_speech.config
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss
import tensorflow as tf

In [5]:
hifigan_config = malaya_speech.config.hifigan_config_v2
hifigan_config['hifigan_generator_params']['filters'] = 512

In [6]:
generator = hifigan.MultiGenerator(
    hifigan.GeneratorConfig(**hifigan_config["hifigan_generator_params"]),
    name="hifigan_generator",
)

In [7]:
multiperiod_discriminator = hifigan.MultiPeriodDiscriminator(
    hifigan.DiscriminatorConfig(**hifigan_config["hifigan_discriminator_params"]),
    name="hifigan_multiperiod_discriminator",
)
multiscale_discriminator = melgan.MultiScaleDiscriminator(
    melgan.DiscriminatorConfig(
        **hifigan_config["melgan_discriminator_params"],
        name="melgan_multiscale_discriminator",
    )
)

In [8]:
discriminator = hifigan.Discriminator(multiperiod_discriminator, multiscale_discriminator)

In [7]:
y = tf.placeholder(tf.float32, (None, None))
x = tf.placeholder(tf.float32, (None, None, 80))
y_hat = generator(x)
# p_hat = discriminator(y_hat)
# p = discriminator(tf.expand_dims(y, -1))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
y_hat

<tf.Tensor 'hifigan_generator/sequential/activation/Tanh:0' shape=(?, ?, 1) dtype=float32>

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [11]:
import numpy as np

sess.run(y_hat, feed_dict = {x: np.random.uniform(size=(1,200,80))}).shape

(1, 51200, 1)

In [12]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [13]:
!ls -lh test
!rm -rf test

total 65256
-rw-r--r--  1 huseinzolkepli  staff    77B Nov  4 13:20 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    31M Nov  4 13:20 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff   3.7K Nov  4 13:20 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   815K Nov  4 13:20 model.ckpt.meta


In [14]:
tf.global_variables()

[<tf.Variable 'hifigan_generator/sequential/conv1d/kernel:0' shape=(7, 80, 512) dtype=float32>,
 <tf.Variable 'hifigan_generator/sequential/conv1d/bias:0' shape=(512,) dtype=float32>,
 <tf.Variable 'hifigan_generator/sequential/conv_transpose_._0/conv2d_transpose/kernel:0' shape=(16, 1, 256, 512) dtype=float32>,
 <tf.Variable 'hifigan_generator/sequential/conv_transpose_._0/conv2d_transpose/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'hifigan_generator/sequential/multi_hifigan_resblock_._0/hifigan_resblock_._0/conv1d_1/kernel:0' shape=(3, 256, 256) dtype=float32>,
 <tf.Variable 'hifigan_generator/sequential/multi_hifigan_resblock_._0/hifigan_resblock_._0/conv1d_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'hifigan_generator/sequential/multi_hifigan_resblock_._0/hifigan_resblock_._0/conv1d_2/kernel:0' shape=(3, 256, 256) dtype=float32>,
 <tf.Variable 'hifigan_generator/sequential/multi_hifigan_resblock_._0/hifigan_resblock_._0/conv1d_2/bias:0' shape=(256,) dtype=float32>,